In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from census import Census
import sqlite3 as sql
import pandas as pd

# Census & gmaps API Keys
from config import (api_key)
c = Census(api_key)

In [2]:
# Retrieve data from census API
census_data = c.acs5.get(("NAME", "B01003_001E", "B05003_008E", "B05003_019E",
                          "B17001_002E", "B02001_002E", "B02001_003E",
                          "B02001_004E", "B02001_005E", "B02001_006E"), 
                         {'for': 'state:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "population",
                                      "B05003_008E": "population_mal_18",
                                      "B05003_019E": "population_fem_18",
                                      "B17001_002E": "poverty_count",
                                      "B02001_002E": "pop_white",
                                      "B02001_003E": "pop_black_afr_american",
                                      "B02001_004E": "pop_american_indian_alaska",
                                      "B02001_005E": "pop_asian",
                                      "B02001_006E": "pop_native_hawaiian_pacific_islands",
                                      "NAME": "state", "state":"number"})

# Add in Population 18+
census_pd["population_18"] = \
    census_pd["population_mal_18"].astype(
        int) + census_pd["population_fem_18"].astype(int)

# Add in Poverty Rate (Poverty Count / Population)
census_pd["poverty_rate"] = 100 * \
    census_pd["poverty_count"].astype(
        int) / census_pd["population"].astype(int)

# Add in Population Race Sum
census_pd["pop_race_sum"] = \
    census_pd["pop_white"].astype(
        int) + census_pd["pop_black_afr_american"].astype(
            int) + census_pd["pop_american_indian_alaska"].astype(
                int) + census_pd["pop_asian"].astype(
                    int) + census_pd["pop_native_hawaiian_pacific_islands"].astype(int)

# Final DataFrame
census_pd = census_pd[["state", "population", "population_mal_18", "population_fem_18", "population_18",
                       "poverty_count", "poverty_rate", "pop_race_sum", "pop_white", "pop_black_afr_american",
                       "pop_american_indian_alaska", "pop_asian", "pop_native_hawaiian_pacific_islands"]]

# Visualize
print(len(census_pd))

census_pd.head()

census_pd.dtypes

52


state                                   object
population                             float64
population_mal_18                      float64
population_fem_18                      float64
population_18                            int32
poverty_count                          float64
poverty_rate                           float64
pop_race_sum                             int32
pop_white                              float64
pop_black_afr_american                 float64
pop_american_indian_alaska             float64
pop_asian                              float64
pop_native_hawaiian_pacific_islands    float64
dtype: object

In [3]:
census_pd['population'] = census_pd['population'].astype(int)
census_pd['population_mal_18'] = census_pd['population_mal_18'].astype(int)
census_pd['population_fem_18'] = census_pd['population_fem_18'].astype(int)
census_pd['poverty_count'] = census_pd['poverty_count'].astype(int)
census_pd['pop_white'] = census_pd['pop_white'].astype(int)
census_pd['pop_black_afr_american'] = census_pd['pop_black_afr_american'].astype(int)
census_pd['pop_american_indian_alaska'] = census_pd['pop_american_indian_alaska'].astype(int)
census_pd['pop_asian'] = census_pd['pop_asian'].astype(int)
census_pd['pop_native_hawaiian_pacific_islands'] = census_pd['pop_native_hawaiian_pacific_islands'].astype(int)

In [4]:
census_pd.dtypes

state                                   object
population                               int32
population_mal_18                        int32
population_fem_18                        int32
population_18                            int32
poverty_count                            int32
poverty_rate                           float64
pop_race_sum                             int32
pop_white                                int32
pop_black_afr_american                   int32
pop_american_indian_alaska               int32
pop_asian                                int32
pop_native_hawaiian_pacific_islands      int32
dtype: object

In [5]:
president_county_candidate = pd.read_csv("president_county_candidate.csv")

In [6]:
conn = sql.connect("usa_elections_2020.sqlite")

In [7]:
census_pd.to_sql('census', conn)
president_county_candidate.to_sql("president_county_candidate", conn)